<a href="https://colab.research.google.com/github/parkgeunyoung1/test/blob/master/(%EC%8B%A4%EC%8A%B5_%EB%AC%B8%EC%A0%9C)4_2(2)_%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8_%EC%84%9C%EB%B9%84%EC%8A%A4_%EB%B0%8F_%EC%8B%9C%EA%B0%81%ED%99%94_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Content License Agreement**

<font color='red'><b>**WARNING**</b></font> : 본 자료는 삼성청년SW·AI아카데미의 컨텐츠 자산으로, 보안서약서에 의거하여 어떠한 사유로도 임의로 복사, 촬영, 녹음, 복제, 보관, 전송하거나 허가 받지 않은 저장매체를 이용한 보관, 제3자에게 누설, 공개 또는 사용하는 등의 무단 사용 및 불법 배포 시 법적 조치를 받을 수 있습니다.

# 환경 설정

## UPSTAGE Credit 및 API Key 발급 받기

1. 회원 가입 진행
  1. <a href = "https://console.upstage.ai/">업스테이지 콘솔</a> 에 방문합니다.
  2. 계정이 없다면, 구글 계정을 통해 회원가입을 진행합니다
  3.  계정에 로그인 합니다.

2. API Key 발급
  1. <a href = "https://console.upstage.ai/api-keys">업스테이지 콘솔 - API Keys</a>페이지를 클릭합니다.
  2. Create New key를 누르고, 발급받은 API key를 복사합니다.
  3. 하단 셀을 실행한 후, 복사한 API Key를 넣습니다.
  4. 세션 재시작시에는 업스테이지 api 를 사용하는 코드를 사용하기 위해 반드시 다시 설정해야 합니다.



In [1]:
# @title set API key
import os
import getpass
import warnings

warnings.filterwarnings("ignore")

# Get the Upstage API key using getpass
if "UPSTAGE_API_KEY" not in os.environ or not os.environ["UPSTAGE_API_KEY"]:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

print("API key has been set successfully.")


Enter your Upstage API key: ··········
API key has been set successfully.


# **Prerequisites**

코랩에서 기본적으로 설치된 라이브러리와 새로 설치하는 라이브리리 사이에서 발생하는 의존성 문제입니다. 이는 Python 패키지 관리 구조상, 특정 버전 조합이 완벽히 호환되지 않는 경우가 많기에 발생하고, 강의 실습에 큰 영향을 주지 않는 단순 Error이니 안심하고 실습을 진행해주셔도 됩니다.

In [2]:
!pip install gradio -q
!pip install langchain_community
!pip install langchain
!pip install langchain_upstage
!pip install langchain_core

  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
Using cached langchain_core-0.3.79-py3-none-any.whl (449 kB)
Using cached langchain_text_splitters-0.3.11-py3-none-any.whl (33 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.1
    Uninstalling langchain-core-1.0.1:
      Successfully uninstalled langchain-core-1.0.1
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 1.0.0
    Uninstalling langchain-text-splitters-1.0.0:
      Successfully uninstalled langchain-text-splitters-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.79 which is incompatible.
langchain

## AI Agent 실전 응용: Gradio로 나만의 챗봇 만들기 실습 가이드

### 실습 개요

본 실습은 지난 차시에서 개발한 '민원 해결사 CS Agent'를 기반으로, Python 웹 프레임워크인 Gradio를 활용하여 사용자와 실시간으로 소통하는 인터랙티브 웹 챗봇 인터페이스를 구축하는 과정입니다. 터미널 환경에서 벗어나 웹 UI를 통해 Agent의 기능을 활용하고, 사용자와의 상호작용 및 Agent의 '생각 과정'을 시각화함으로써 AI Agent가 실제 서비스로 구현되는 방식을 경험합니다. 가상의 'AI 온라인 서점' 시나리오를 통해 실제 서비스 개발과 유사한 경험을 제공합니다.

### 실습 진행 목적 및 배경

최근 AI 기술의 발전과 함께 Agent 기반 시스템의 활용이 늘어나고 있습니다. 하지만 Agent가 사용자에게 직접 서비스를 제공하기 위해서는 직관적이고 사용하기 쉬운 인터페이스가 필수적입니다. 본 실습은 사용자들이 Agent와 쉽게 소통할 수 있는 웹 기반 챗봇 인터페이스를 Gradio로 빠르게 구축하고, Agent의 내부 '생각 과정(Thought)' 및 구조화된 출력을 사용자에게 투명하게 보여줌으로써 AI 시스템에 대한 신뢰를 높이는 방법을 학습하는 데 목적이 있습니다.

### 실습 수행으로 얻어갈 수 있는 역량

*   **Gradio를 활용한 웹 UI 개발 능력:** Gradio의 핵심 컴포넌트(`gr.Chatbot`, `gr.Textbox`, `gr.JSON`, `gr.Blocks`)를 사용하여 데이터 기반 애플리케이션 및 챗봇과 같은 인터랙티브 웹 인터페이스를 빠르게 구축하는 능력을 습득합니다.
*   **AI Agent와 웹 UI 연동 경험:** 개발된 AI Agent 로직을 웹 프론트엔드와 연결하여 서비스 가능한 형태로 만드는 실무 경험을 쌓습니다. Gradio의 이벤트 핸들링(`.click()`, `.submit()`)을 사용하여 사용자 입력과 Agent 호출을 연결하는 방법을 배웁니다.
*   **Agent의 투명성 확보 방법 학습:** Agent의 추론 과정(Thought, Action, Observation) 및 최종 구조화된 출력(JSON/MCP)을 Gradio 컴포넌트를 통해 시각적으로 제공하여 시스템의 신뢰성을 높이는 방법을 이해하고 적용합니다. Gradio의 `yield`를 사용하여 비동기적인 UI 업데이트를 구현하는 방법을 익힙니다.
*   **Gradio Blocks를 사용한 레이아웃 구성:** `gr.Blocks`를 사용하여 여러 Gradio 컴포넌트를 조합하고 복잡한 레이아웃을 구성하여 챗봇과 부가 정보(Agent 생각 과정, JSON 출력)를 함께 표시하는 방법을 학습합니다.

### 실습 핵심 내용

*   Gradio 기본 UI 컴포넌트 (`gr.Chatbot`, `gr.Textbox`, `gr.Blocks`, `gr.JSON`) 활용
*   Agent 로직을 Gradio 애플리케이션으로 `import` 및 연동
*   Python 함수 내에서 Agent 로딩 및 캐싱 구현
*   사용자 입력에 따라 Agent 호출 및 응답 생성 (`agent.invoke()`)
*   Agent의 '생각 과정' (Intermediate Steps) 파싱 및 `gr.Textbox`로 시각화
*   Agent의 최종 출력 중 JSON 형식을 `gr.JSON`으로 시각화
*   Gradio `Blocks` 및 이벤트 핸들링(`.click()`, `.submit()`)을 사용한 UI 로직 연결
*   Gradio `yield`를 사용한 UI 업데이트
*   Gradio 애플리케이션 실행 및 Colab 환경에서의 외부 접근 (`share=True`)

* * *

## AI Agent 실전 응용: Gradio로 나만의 챗봇 만들기 실습 목차

### 전체 목차

*   **실습 1: 챗봇의 '얼굴' 만들기 (Gradio 기본 UI 구성)**
    *   Gradio 기본 구조 이해 (`gr.ChatInterface` 또는 `gr.Blocks`)
    *   챗봇 메시지 표시 영역 (`gr.Chatbot`) 및 사용자 입력창 (`gr.Textbox`) 구성
    *   `gr.ChatInterface`의 기본 대화 관리 방식 이해
*   **실습 2: Agent의 '두뇌'와 '얼굴' 연결하기 (Agent 연동)**
    *   이전 실습 Agent 로직 `import` (`CustomerServiceAgent` 클래스)
    *   Python 함수 내에서 Agent 객체 캐싱 로딩 구현
    *   Gradio 이벤트 핸들러(예: `.submit()`, `.click()`)에서 Agent 호출 (`agent.invoke()`)
    *   Agent 응답을 Gradio 챗봇에 표시

* **실습 내용: 완성된 챗봇의 작동 모습을 시연하고 결과물을 확인합니다.**
    * 챗봇을 실행합니다.
    * "안녕하세요?" 같은 간단한 인사로 시작합니다.
    * "책 배송이 너무 늦어요. 주문 12345 취소하고 보상해주세요. 그리고 쿠폰도 주세요."와 같은 복합적인 민원을 입력합니다.
    * Agent가 실시간으로 생각하는 과정(st.status)과 최종 MCP(st.json) 또는 답변을 출력하는 전체 과정을 확인합니다.

* **챗봇 개선 및 확장 (선택 사항)**
    *   사용자 경험(UX) 개선 (예: 로딩 인디케이터 커스터마이징)
    *   에러 처리 강화 및 UI 피드백 개선
    *   더 복잡한 자연어 처리 모델 연동
    *   Gradio 애플리케이션 배포 맛보기

## 실습 1: 챗봇의 '얼굴' 만들기 (Gradio 기본 UI 구성)

본 실습에서는 Gradio의 핵심 컴포넌트를 사용하여 챗봇 애플리케이션의 기본적인 사용자 인터페이스를 구축합니다. 사용자와 챗봇 메시지를 표시할 영역을 만들고, 사용자의 입력을 받는 입력창을 구현합니다. Gradio의 `gr.ChatInterface`를 사용하면 기본적인 대화 UI와 상태 관리가 자동으로 처리됩니다. 또는 `gr.Blocks`를 사용하여 컴포넌트 레이아웃을 직접 제어할 수도 있습니다. 실습 1에서는 `gr.ChatInterface`를 사용하여 간단히 구현하고, 이후 실습에서 `gr.Blocks`로 전환하여 추가 시각화 컴포넌트를 배치할 것입니다.

### 학습 키워드 설명

*   **`Gradio`**: Python 함수를 몇 줄의 코드로 공유 가능한 웹 UI로 빠르게 변환할 수 있는 오픈 소스 라이브러리입니다. 머신러닝 모델의 데모 구축에 널리 사용되지만, 간단한 웹 서비스나 챗봇 UI를 만드는 데도 매우 유용합니다.
*   **`Web Interface`**: 사용자가 소프트웨어 시스템과 상호작용하기 위해 사용하는 시각적인 요소와 컨트롤(버튼, 입력 필드, 텍스트 등)의 집합입니다. 웹 브라우저를 통해 접근 가능하며, 사용자의 입력을 받고 시스템의 결과를 표시하는 역할을 합니다. 챗봇의 경우, 사용자가 메시지를 입력하고 챗봇의 답변을 볼 수 있는 화면이 웹 인터페이스입니다.
*   **`gr.ChatInterface`**: Gradio에서 챗봇 애플리케이션을 쉽게 구축할 수 있도록 미리 구성된 인터페이스 컴포넌트입니다. `fn` 인자에 사용자 입력 처리 함수를 연결하면, 메시지 입력, 대화 기록 표시, 히스토리 관리 등의 기본적인 챗봇 기능을 자동으로 제공합니다.
*   **`gr.Chatbot`**: 챗봇 대화 기록을 표시하는 데 사용되는 Gradio 컴포넌트입니다. `gr.ChatInterface` 내부에 포함되어 사용되거나, `gr.Blocks`에서 독립적으로 사용될 수 있습니다. 사용자 메시지와 챗봇 메시지를 구분하여 표시합니다.
*   **`gr.Textbox`**: 사용자가 텍스트를 입력하거나 텍스트를 출력하는 데 사용되는 Gradio 컴포넌트입니다. 챗봇 인터페이스에서는 주로 사용자 입력창으로 활용됩니다. `gr.ChatInterface` 내부에 입력창으로 포함되거나, `gr.Blocks`에서 독립적으로 배치될 수 있습니다.
*   **`history` 인자 (in Gradio functions)**: `gr.ChatInterface`의 `fn`에 연결된 함수는 첫 번째 인자로 사용자 입력 메시지(string)를 받고, 두 번째 인자로 현재까지의 대화 기록(list of `[user_message, bot_message]` 쌍)을 받습니다. 함수는 업데이트된 대화 기록을 반환해야 합니다.

### 실습 방식

1.  **`app.py` 파일 수정:** 기존에 생성된 `app.py` 파일의 내용을 아래 제공된 템플릿 코드로 변경합니다.
2.  **`TODO` 위치에 코드 작성:** 템플릿 코드 내에 표시된 `TODO` 위치에 맞춰 다음 코드를 작성합니다.

In [3]:
import gradio as gr

# TODO 3 : 이전 대화 기록 및 응답 설정
def chat(message, history):
    turns = len(history) if history else 0
    return f"(이전 대화 {turns}턴) 방금 하신 말씀: {message}"

gr.ChatInterface(chat,
                #TODO 1 : 제목설정
                title="간단 에코 챗봇",
                #TODO 2 :초기 메세지 설정
                description="안녕하세요! 메시지를 입력하면 이전 대화 턴 수와 함께 그대로 돌려드립니다 🙂").launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91bc0908bd6c4d2720.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



## 실습 2: Agent의 '두뇌'와 '얼굴' 연결하기 (Agent 연동)

본 실습에서는 이전 차시에서 개발한 '민원 해결사 Agent'의 로직을 Gradio 애플리케이션에 통합합니다. 사용자가 챗봇 UI를 통해 질문을 입력하면, Gradio 앱의 이벤트 핸들러가 이 입력을 감지하여 Agent 호출 함수를 실행하고, Agent가 생성한 답변을 받아 다시 Gradio 챗봇 컴포넌트에 표시하도록 연결합니다. Agent 객체는 로딩 시간이 오래 걸릴 수 있으므로, 애플리케이션 실행 중 한 번만 로드되도록 Python 함수 내에서 캐싱하는 방법을 사용합니다.

### 학습 키워드 설명

*   **`Agent 기능 연동`**: 별도로 구현된 AI Agent의 핵심 로직(여기서는 `CustomerServiceAgent` 클래스의 인스턴스)을 다른 애플리케이션(여기서는 Gradio 웹 앱)에 통합하여 사용하는 과정입니다.
*   **`import`**: Python 모듈 시스템에서 다른 파일이나 패키지에 정의된 함수, 클래스, 변수 등을 현재 파일로 가져와 사용할 수 있게 하는 구문입니다. 이전 실습에서 작성한 Agent 코드를 `app.py`로 가져오는 데 사용합니다.
*   **`Python 함수 내 캐싱`**: Gradio 앱이 다시 로드될 때 Agent 객체가 불필요하게 여러 번 생성되는 것을 방지하여 성능을 최적화하기 위해, 일반 Python 함수 스코프 또는 전역 변수를 활용하여 객체를 한 번만 생성하고 재사용하는 기법입니다.
*   **`gr.ChatInterface`의 `fn` 인자**: `gr.ChatInterface`를 사용할 때, 사용자가 메시지를 입력하고 제출할 때 호출될 Python 함수를 지정합니다. 이 함수는 사용자 메시지와 현재 대화 기록을 인자로 받고, 업데이트된 대화 기록을 반환해야 합니다.
*   **`agent.invoke()`**: LangChain Agent Executor 객체의 메서드입니다. 사용자 입력(프롬프트)을 인자로 받아 Agent의 실행을 시작하고, Agent가 작업을 완료한 후 최종 결과(답변, 중간 단계 등)를 반환합니다. Gradio 앱에서는 사용자 입력이 발생했을 때 `fn`에 연결된 함수 내에서 이 메서드를 호출하여 Agent의 답변을 얻습니다.
*   **`result['output']`**: `agent.invoke()` 호출 결과로 반환되는 딕셔너리에서 Agent의 최종 답변 텍스트를 포함하는 키입니다. 이 값을 추출하여 챗봇 응답으로 사용자에게 표시합니다.

### 실습 방식

1.  **Agent 코드 준비:** 이전 차시에서 완성한 '민원 해결사 Agent'의 Python 코드가 `cs_agent.py` 파일로 저장되어 있는지 확인합니다. 이 파일에는 `CustomerServiceAgent` 클래스가 포함되어 있어야 합니다.
2.  **Agent 클래스 import:** `from cs_agent import CustomerServiceAgent`와 같이 Agent 클래스를 import 하는 코드를 `TODO 1` 위치에 작성합니다.
3.  **Python 함수 내에서 Agent 객체 캐싱 구현:** `CustomerServiceAgent` 클래스의 인스턴스를 생성하고 반환하는 함수(`get_agent_instance`)를 정의합니다. 이 함수는 전역 변수 등을 활용하여 Agent 인스턴스가 이미 생성되었는지 확인하고, 생성되지 않았다면 새로 만들고 저장한 후 반환하도록 구현합니다. 이 코드를 `TODO 2` 위치에 완성합니다.
4.  **`gr.ChatInterface`의 `fn` 함수 수정:** 실습 1에서 작성한 `respond` 함수를 수정하여, 사용자 입력(`message`)이 발생했을 때 `get_agent_instance()`를 호출하여 Agent 객체를 가져오고, `agent.invoke({"input": message})`를 호출하여 Agent의 응답(`result`)을 받습니다. 이 코드를 `TODO 3` 위치에 작성합니다.
5.  **Agent 응답을 대화 기록에 추가:** `agent.invoke()` 호출 결과에서 최종 답변(`result['output']`)을 추출하여, Gradio의 `history` 형식([user_message, bot_message] 쌍의 리스트)에 맞게 새로운 메시지 쌍을 추가하고 업데이트된 `history`를 반환하는 코드를 `TODO 4` 위치에 완성합니다.

#### `reward_policy.txt` 파일 준비

YES24의 정책에 관한 PDF 문서에서 보상 정책만을 텍스트로 추출합니다.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import glob # 파일을 검색하기 위한 glob 모듈을 가져옵니다.
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyMuPDFLoader

# './' 경로에 있는 모든 PDF 파일을 찾습니다.
pdf_files = glob.glob('/content/drive/*.pdf')

all_documents = [] # 모든 PDF 파일에서 로드된 문서를 저장할 빈 리스트를 생성합니다.

print(f"--- './' 경로에서 찾은 PDF 파일 목록: {pdf_files} ---")

# 찾은 각 PDF 파일에 대해 로드 및 파싱을 수행합니다.
for pdf_filepath in pdf_files:
    try:
        #loader = PyPDFLoader(pdf_filepath)
        loader = PyMuPDFLoader(pdf_filepath)
        pages = loader.load()
        all_documents.extend(pages) # 로드된 페이지들을 all_documents 리스트에 추가합니다.
        print(f"'{pdf_filepath}' 파일 로드 완료. {len(pages)} 페이지 로드됨.")
    except Exception as e:
        print(f"'{pdf_filepath}' 파일 로드 중 오류 발생: {e}")

# shipping_policy.txt 파일도 로드하여 all_documents에 추가합니다.
shipping_policy_file_path = './shipping_policy.txt'

try:
    # Create a TextLoader instance with the file path
    loader = TextLoader(shipping_policy_file_path)

    # Load the document
    text_documents = loader.load()

    # Extend all_documents with text file documents
    all_documents.extend(text_documents)
    print(f"'{shipping_policy_file_path}' 파일 로드 완료. {len(text_documents)} 문서 로드됨.")


except FileNotFoundError:
    print(f"경로에 파일을 찾을 수 없습니다{shipping_policy_file_path}. ")
except Exception as e:
    print(f"해당 경로에 오류가 발생했습니다{shipping_policy_file_path}: {e}")


# 모든 로드된 총 문서(PDF 페이지 + Text 문서) 정보 출력
print("\n--- 모든 파일에서 로드된 총 문서 정보 ---")
print(f"총 문서 개수: {len(all_documents)}")
if all_documents:
    print(f"첫 번째 문서 내용 (일부): {all_documents[0].page_content[:200]}...")
    print(f"첫 번째 문서 메타데이터: {all_documents[0].metadata}")
else:
    print("로드된 문서가 없습니다.")
print("-" * 20)

# 다음 단계를 위해 변수 이름을 'documents'로 맞춥니다.
documents = all_documents

--- './' 경로에서 찾은 PDF 파일 목록: [] ---
'./shipping_policy.txt' 파일 로드 완료. 1 문서 로드됨.

--- 모든 파일에서 로드된 총 문서 정보 ---
총 문서 개수: 1
첫 번째 문서 내용 (일부): # AI 온라인 서점 배송 정책

## 일반 배송
- 평일 오후 3시 이전 주문 시 당일 발송됩니다.
- 오후 3시 이후 주문 건은 익일 발송됩니다.
- 주말 및 공휴일은 배송이 어렵습니다.

## 도서 산간 지역 배송
- 제주 및 도서 산간 지역은 추가 배송비가 발생할 수 있습니다.
- 추가 배송비 및 예상 소요 시간은 주문 시 확인 가능합니다.

## ...
첫 번째 문서 메타데이터: {'source': './shipping_policy.txt'}
--------------------


In [6]:
%%writefile shipping_policy.txt
# AI 온라인 서점 배송 정책

## 일반 배송
- 평일 오후 3시 이전 주문 시 당일 발송됩니다.
- 오후 3시 이후 주문 건은 익일 발송됩니다.
- 주말 및 공휴일은 배송이 어렵습니다.

## 도서 산간 지역 배송
- 제주 및 도서 산간 지역은 추가 배송비가 발생할 수 있습니다.
- 추가 배송비 및 예상 소요 시간은 주문 시 확인 가능합니다.

## 배송 조회
- 주문 번호 order-123의 배송 상태는 마이페이지에서 조회 가능합니다.
- 회원 및 비회원 모두 주문 번호로 배송 조회가 가능합니다.
- 배송 관련 문의는 고객센터로 연락 주시기 바랍니다.

Overwriting shipping_policy.txt


#### CS 에이전트 준비

지난 단계에서 개발한 CS 에이전트를 준비합니다

In [7]:
%%writefile cs_agent.py
# 필요한 라이브러리 임포트
import os
from typing import Any, Dict
import warnings

# LangChain 관련 임포트
from langchain.agents import create_react_agent, AgentExecutor, Tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_community.vectorstores import Chroma

warnings.filterwarnings("ignore")

class CustomerServiceAgent:
    """
    AI 온라인 서점 민원 해결사 Agent 클래스.
    Agent 및 필요한 구성 요소를 초기화하고 대화 기능을 제공합니다.
    """
    def __init__(self, api_key: str | None = None):
        """Agent 및 필요한 구성 요소를 초기화하고 환경 설정을 수행합니다."""
        print("\n--- CustomerServiceAgent 초기화 시작 ---")

        # (1) API Key 설정
        if api_key and not os.environ.get("UPSTAGE_API_KEY"):
            os.environ["UPSTAGE_API_KEY"] = api_key

        # 1. LLM
        self.llm = self._create_llm()
        if self.llm is None:
            warnings.warn("LLM 로드 실패. Agent가 제대로 작동하지 않을 수 있습니다.")
            self.agent_executor = None
            return

        # 2. RAG
        self.retriever = self._setup_rag_vectorstore()
        if self.retriever is None:
            warnings.warn("RAG Vector Store 설정 실패. 정책 검색 도구가 비활성화됩니다.")

        # 3. 도구
        self.tools = self._define_agent_tools(self.retriever)
        if not self.tools:
            warnings.warn("Agent 도구가 정의되지 않았습니다. Agent가 작동할 수 없습니다.")
            self.agent_executor = None
            return

        # 4. 메모리
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

        # 5. 프롬프트 (chat_history 반영!)
        self.prompt_template = ChatPromptTemplate.from_messages([
            ("system", self._get_system_prompt()),
            ("system", "이전 대화:\n{chat_history}"),
            ("human", "{input}\n\n{agent_scratchpad}"),
        ])

        # 6. Agent Executor
        self.agent_executor = self._create_agent_executor(
            self.llm, self.tools, self.prompt_template, self.memory
        )

        if self.agent_executor:
            print("CustomerServiceAgent 초기화 완료.")
        else:
            print("경고: CustomerServiceAgent 초기화 실패!")

    # -------------------- 유틸/도구 --------------------
    def _get_order_status(self, order_id: str):
        print(f"\n[DEBUG] _get_order_status 호출: {order_id}")
        if order_id == "ORDER123": return "Delivered"
        elif order_id == "ORDER456": return "Shipping Delayed"
        elif order_id == "ORDER789": return "Processing"
        else: return "Order Not Found"

    def _issue_complaint_coupon(self, order_id: str):
        print(f"\n[DEBUG] _issue_complaint_coupon 호출: {order_id}")
        order_status = self._get_order_status(order_id)
        if order_status == "Shipping Delayed":
            print(f"모의: 지연된 주문 ID {order_id}에 대해 쿠폰 발급 완료.")
            return f"주문 ID {order_id}에 대해 5,000원 할인 쿠폰이 발급되었습니다."
        else:
            print(f"모의: 주문 ID {order_id} 상태 '{order_status}'는 쿠폰 발급 대상이 아닙니다.")
            return f"주문 ID {order_id}는 쿠폰 발급 대상이 아닙니다. 현재 상태: {order_status}"

    # -------------------- RAG --------------------
    def _setup_rag_vectorstore(self):
        pdf_files = [
            './신규 회원 혜택 | 서비스:혜택 - 예스24.pdf',
            './매장 픽업 서비스 | 서비스:혜택 - 예스24.pdf',
            './도서 품절 보상제도 | 예스24.pdf',
            './무료배송+추가적립 | 서비스:혜택 - 예스24.pdf',
            './배송지연 보상제도 | 서비스:혜택 - 예스24.pdf',
            './총알배송 | 서비스:혜택 - 예스24.pdf',
            './영원한 YES포인트 | 서비스:혜택 - 예스24.pdf',
            './제휴 할인카드 | 서비스:혜택 - 예스24.pdf'
        ]
        shipping_policy_file_path = './shipping_policy.txt'

        all_docs = []
        try:
            # PDF 로드
            for pdf_filepath in pdf_files:
                if os.path.exists(pdf_filepath):
                    try:
                        loader = PyMuPDFLoader(pdf_filepath)
                        pages = loader.load()
                        all_docs.extend(pages)
                        print(f"'{pdf_filepath}' 로드 완료. {len(pages)} 페이지 로드됨.")
                    except Exception as e:
                        print(f"경고: '{pdf_filepath}' 파일 로드 중 오류 발생: {e}")
                else:
                    print(f"경고: '{pdf_filepath}' 파일을 찾을 수 없습니다.")

            # Text 로드
            if os.path.exists(shipping_policy_file_path):
                try:
                    shipping_loader = TextLoader(shipping_policy_file_path, encoding='utf-8')
                    all_docs.extend(shipping_loader.load())
                    print(f"'{shipping_policy_file_path}' 로드 완료.")
                except Exception as e:
                    print(f"경고: '{shipping_policy_file_path}' 파일 로드 중 오류 발생: {e}")

            refund_policy_file_path = './refund_policy.txt'
            if os.path.exists(refund_policy_file_path):
                try:
                    refund_loader = TextLoader(refund_policy_file_path, encoding='utf-8')
                    all_docs.extend(refund_loader.load())
                    print(f"'{refund_policy_file_path}' 로드 완료.")
                except Exception as e:
                    print(f"경고: '{refund_policy_file_path}' 파일 로드 중 오류 발생: {e}")
            else:
                print(f"경고: '{refund_policy_file_path}' 파일을 찾을 수 없습니다. RAG에 포함되지 않습니다.")

            if not all_docs:
                print("경고: RAG 문서를 로드하지 못했습니다. RAG 도구가 비활성화됩니다.")
                return None

            text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
            split_docs = text_splitter.split_documents(all_docs)
            print(f"총 {len(all_docs)}개 문서 -> {len(split_docs)}개 청크로 분할.")

            embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
            vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)
            retriever = vectorstore.as_retriever()
            print("RAG Vector Store (Chroma) 설정 및 Retriever 준비 완료.")
            return retriever

        except Exception as e:
            print(f"RAG 설정 중 오류 발생: {e}")
            return None

    def _define_agent_tools(self, retriever):
        agent_tools = []

        if retriever:
            rag_tool = Tool(
                name="Policy Search",
                func=lambda q: "\n\n".join(
                    [f"[{i+1}] {d.page_content[:800]}" for i,d in enumerate(retriever.get_relevant_documents(q))]
                ),
                description="배송/환불 정책에서 관련 정보를 검색합니다. 입력: 자연어 질문."
            )
            agent_tools.append(rag_tool)
            print("'Policy Search' 도구 추가 완료.")
        else:
            print("경고: RAG Retriever가 없어 'Policy Search' 도구를 추가할 수 없습니다.")

        order_status_tool = Tool(
            name="Get Order Status",
            func=self._get_order_status,
            description="주문 ID의 현재 배송 상태를 조회합니다. 입력: 주문 ID (예: ORDER123)."
        )
        agent_tools.append(order_status_tool)
        print("'Get Order Status' 도구 추가 완료.")

        coupon_tool = Tool(
            name="Issue Complaint Coupon",
            func=self._issue_complaint_coupon,
            description="배송 지연 주문(예: ORDER456)에 대해 불만 쿠폰을 발급합니다. 입력: 주문 ID."
        )
        agent_tools.append(coupon_tool)
        print("'Issue Complaint Coupon' 도구 추가 완료.")

        print(f"\n구성된 Agent 도구 목록: {[tool.name for tool in agent_tools]}")
        return agent_tools

    def _create_llm(self):
        try:
            api_key = os.environ.get("UPSTAGE_API_KEY")
            if not api_key:
                print("경고: UPSTAGE_API_KEY 환경 변수가 설정되지 않았습니다.")
                return None
            llm = ChatUpstage(upstage_api_key=api_key)
            print(f"\nUpstage LLM ({llm.model_name}) 준비 완료.")
            return llm
        except Exception as e:
            print(f"\nUpstage LLM 인스턴스 생성 중 오류 발생: {e}")
            return None

    def _create_agent_executor(self, llm, tools, prompt_template, memory):
        if llm is None or not tools or prompt_template is None or memory is None:
            print("경고: Agent Executor 생성 요건 누락.")
            return None

        agent = create_react_agent(llm, tools, prompt_template)
        print("\nAgent 생성 완료.")

        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            memory=memory,
            handle_parsing_errors=True
        )
        print("Agent Executor 구성 완료 (메모리 연결).")
        return agent_executor

    def _get_system_prompt(self):
        return """당신은 친절하고 유능한 고객 서비스 에이전트입니다.
도구:
{tools}

ReAct 형식으로 추론/행동/관찰을 반복하세요.

형식:
Thought: ...
Action: 도구이름[입력값]
Observation: ...
... 반복 ...
Thought: 충분한 정보가 모였습니다.
Final Answer: (최종 답변)

도구 목록: {tool_names}
이전 대화 기록을 고려해 응답하세요.
"""

    def invoke(self, user_input: str, config: Dict[str, Any] = None) -> Dict[str, Any]:
        if self.agent_executor is None:
            print("오류: Agent Executor가 초기화되지 않았습니다.")
            return {"output": "Agent 시스템 오류가 발생했습니다. 초기화 로그를 확인해주세요."}
        try:
            result = self.agent_executor.invoke({"input": user_input}, config=config)
            return result
        except Exception as e:
            print(f"Agent 실행 중 오류 발생: {e}")
            return {"output": f"Agent 실행 중 오류 발생: {e}", "error": str(e)}

    def is_ready(self):
        return self.agent_executor is not None

print("\n에이전트 기능이 CustomerServiceAgent 클래스로 완전히 재구성되었습니다.")


Overwriting cs_agent.py


#### app.py (실습 1 + 실습 2)


In [8]:
from google.colab import userdata
import os

# Colab Secrets에서 불러와 환경변수로 설정
os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")

# 확인
print("🔑 UPSTAGE_API_KEY 설정 완료:", bool(os.environ.get("UPSTAGE_API_KEY")))

🔑 UPSTAGE_API_KEY 설정 완료: True


In [9]:
from cs_agent import CustomerServiceAgent
api_key = os.environ.get("UPSTAGE_API_KEY")
print("API 키 불러오기 성공:", bool(api_key))

agent = CustomerServiceAgent()
print("Agent 준비 상태:", agent.is_ready())


에이전트 기능이 CustomerServiceAgent 클래스로 완전히 재구성되었습니다.
API 키 불러오기 성공: True

--- CustomerServiceAgent 초기화 시작 ---

Upstage LLM (solar-mini) 준비 완료.
경고: './신규 회원 혜택 | 서비스:혜택 - 예스24.pdf' 파일을 찾을 수 없습니다.
경고: './매장 픽업 서비스 | 서비스:혜택 - 예스24.pdf' 파일을 찾을 수 없습니다.
경고: './도서 품절 보상제도 | 예스24.pdf' 파일을 찾을 수 없습니다.
경고: './무료배송+추가적립 | 서비스:혜택 - 예스24.pdf' 파일을 찾을 수 없습니다.
경고: './배송지연 보상제도 | 서비스:혜택 - 예스24.pdf' 파일을 찾을 수 없습니다.
경고: './총알배송 | 서비스:혜택 - 예스24.pdf' 파일을 찾을 수 없습니다.
경고: './영원한 YES포인트 | 서비스:혜택 - 예스24.pdf' 파일을 찾을 수 없습니다.
경고: './제휴 할인카드 | 서비스:혜택 - 예스24.pdf' 파일을 찾을 수 없습니다.
'./shipping_policy.txt' 로드 완료.
경고: './refund_policy.txt' 파일을 찾을 수 없습니다. RAG에 포함되지 않습니다.
총 1개 문서 -> 1개 청크로 분할.
RAG 설정 중 오류 발생: Could not import chromadb python package. Please install it with `pip install chromadb`.
경고: RAG Retriever가 없어 'Policy Search' 도구를 추가할 수 없습니다.
'Get Order Status' 도구 추가 완료.
'Issue Complaint Coupon' 도구 추가 완료.

구성된 Agent 도구 목록: ['Get Order Status', 'Issue Complaint Coupon']

Agent 생성 완료.
Agent Executor 구성 완료 (메모리 연결).
Customer

In [10]:
import gradio as gr
import os
from cs_agent import CustomerServiceAgent

agent_instance = None

def get_agent_instance():
    global agent_instance
    if agent_instance is None:
        print("Agent 인스턴스 로드 중...")
        try:
            api_key = os.environ.get("UPSTAGE_API_KEY")
            if not api_key:
                print("경고: UPSTAGE_API_KEY 환경 변수가 설정되지 않았습니다. Agent 로드 실패.")
                return None
            agent_instance = CustomerServiceAgent(api_key=api_key)
            if not agent_instance.is_ready():
                print("챗봇 Agent 로드 실패. 키/문서 확인 후 재시도.")
                agent_instance = None
                return None
            print("Agent 인스턴스 로드 완료.")
            return agent_instance
        except Exception as e:
            print(f"Agent 인스턴스 로딩 중 오류 발생: {e}")
            agent_instance = None
            return None
    print("캐시된 Agent 인스턴스 사용.")
    return agent_instance

async def process_message(message, chat_history):
    chat_history = (chat_history or []) + [{"role": "user", "content": message}, {"role": "assistant", "content": ""}]
    yield "", chat_history

    agent = get_agent_instance()
    if agent is None or not agent.is_ready():
        err = "죄송합니다. 챗봇 시스템에 문제가 발생했습니다. (Agent 로드 실패)"
        chat_history[-1]["content"] = err
        yield gr.update(value=""), chat_history
        return

    try:
        result = agent.agent_executor.invoke({"input": message})
        agent_response = result.get('output', 'Agent 응답 없음')
        chat_history[-1]["content"] = agent_response
        yield gr.update(value=""), chat_history
    except Exception as e:
        err = f"죄송합니다. 요청 처리 중 오류가 발생했습니다: {e}"
        chat_history[-1]["content"] = err
        yield gr.update(value=""), chat_history

with gr.Blocks(title="AI 온라인 서점 챗봇") as demo:
    gr.Markdown("# AI 온라인 서점 챗봇")
    gr.Markdown("무엇이든 물어보세요!")
    chatbot = gr.Chatbot(height=400, type='messages', label="Chat History")
    with gr.Row():
        textbox = gr.Textbox(placeholder="메시지를 입력하세요...", container=False, scale=7, label="Your Message")
        submit_btn = gr.Button("Submit")

    submit_btn.click(process_message, inputs=[textbox, chatbot], outputs=[textbox, chatbot])
    textbox.submit(process_message, inputs=[textbox, chatbot], outputs=[textbox, chatbot])

demo.launch(inline=True, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://079ac04c2702961667.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 실습 마무리

이번 경험을 통해 다음 키워드들을 중심으로 AI Agent와 웹 UI 연동 실습을 성공적으로 수행했습니다.

*   **Gradio**: Python 함수를 웹 UI로 빠르게 변환할 수 있는 라이브러리입니다. Streamlit 대신 Gradio를 사용하여 챗봇 인터페이스를 구축하고 `gr.ChatInterface` 및 `gr.Blocks`와 같은 핵심 컴포넌트를 활용하는 방법을 배웠습니다.
*   **AI Agent 연동**: 별도로 개발된 `CustomerServiceAgent` 클래스를 Gradio 웹 애플리케이션에 통합하여 사용자 입력에 따라 Agent가 작동하고 응답을 생성하도록 연결했습니다.
*   **Python 함수 캐싱**: Agent 객체처럼 로딩 시간이 오래 걸리는 리소스를 애플리케이션 실행 중 한 번만 로드하여 성능을 최적화하기 위해 Python 함수 내에서 전역 변수를 활용한 캐싱 기법을 적용했습니다.
*   **Agent 생각 과정 시각화**: Agent의 ReAct 추론 과정(Thought, Action, Observation)을 `intermediate_steps`에서 추출하여 `gr.Textbox`에 보기 쉽게 표시함으로써 Agent의 작동 투명성을 높였습니다.
*   **JSON 출력 처리 및 시각화**: Agent의 최종 출력이 JSON 형식일 경우 이를 감지하고 파싱하여 `gr.JSON` 컴포넌트를 통해 구조화된 형태로 보여주는 방법을 구현했습니다.
*   **Gradio Blocks 및 이벤트 핸들링**: `gr.Blocks`를 사용하여 챗봇 창, 생각 과정, JSON 출력 등 여러 컴포넌트를 원하는 레이아웃으로 배치하고, `.click()` 및 `.submit()` 이벤트 핸들러를 사용하여 사용자 인터랙션과 Agent 호출 및 UI 업데이트 로직을 연결했습니다.
*   **Gradio `yield`**: `process_message` 함수 내에서 `yield`를 사용하여 Agent 실행 중 또는 완료 후에 여러 UI 컴포넌트의 상태를 업데이트하는 방법을 경험했습니다.
*   **Colab 환경 배포**: `demo.launch(share=True, inline=True)` 명령어를 사용하여 Colab 환경에서 개발한 Gradio 웹 애플리케이션을 외부에서도 접속할 수 있도록 공유 링크를 생성하는 방법을 확인했습니다.

이 실습을 통해 Agent 시스템이 실제 사용자 서비스와 어떻게 연결되고, Agent의 내부 작동 과정을 사용자에게 어떻게 투명하게 보여줄 수 있는지에 대한 실무적인 경험을 쌓았습니다.